In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
import ipywidgets as ipw
from IPython.display import HTML, display, clear_output

In [ ]:
def delete_nodes(b):
    if not toggle_really.value:
        with out:
            clear_output()
            print('I did not change your database.')
        return
    
    from django.db import transaction
    from django.db.models import Q
    from aiida.backends.djsite.db import models
    from aiida.orm import load_node

    all_pks_to_delete = set([int(x) for x in text_pks.value.split(' ')])
    with out:
        clear_output()
        print 'I will delete {} nodes: {}.'.format(len(all_pks_to_delete), str(all_pks_to_delete))

    # Recover the list of folders to delete before actually deleting
    # the nodes.  I will delete the folders only later, so that if
    # there is a problem during the deletion of the nodes in
    # the DB, I don't delete the folders
    folders = [load_node(pk).folder for pk in all_pks_to_delete]

    with transaction.atomic():
        # Delete all links pointing to or from a given node
        models.DbLink.objects.filter(
            Q(input__in=all_pks_to_delete) |
            Q(output__in=all_pks_to_delete)).delete()
        # now delete nodes
        models.DbNode.objects.filter(pk__in=all_pks_to_delete).delete()

    # If we are here, we managed to delete the entries from the DB.
    # I can now delete the folders
    for f in folders:
        f.erase()
        
    with out:
        clear_output()
        print('All done.')

# Delete nodes

In [ ]:
html = '''
<style>
#dontdoit {
    border: 3px solid #962d22;
    background: rgba(192, 57, 43, 0.9);
    padding: 1em;
    font-size: 1.2em;
    font-weight: bold;
    color: #fafafa;
    text-align: center;
}
</style>
<div id="dontdoit">
    Deleting nodes can break provenance and create orphans.
    Continue only, if you are absolutely sure you know what you are doing! 
</div>
'''
display(HTML(html))

style = {'description_width': '120px'}
layout = {'width': '50%'}
text_pks = ipw.Text(description='Nodes to delete',
                    placeholder='1 2 100 10051',
                    layout=layout, style=style)
toggle_really = ipw.Checkbox(description="I know I shouldn't do this.",
                                 layout=layout, style=style)
button_delete = ipw.Button(description="Do it!",
                           layout=layout, style=style)
button_delete.on_click(delete_nodes)
display(ipw.VBox([text_pks, toggle_really, button_delete]))

out = ipw.Output()
display(out)